# Imports

In [1]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt

from dstoolkit.feature_engine import GroupedLagTimeFeatureCreator

## Loading Dataset

In [2]:
# the data is from https://datahub.io/core/global-temp
df = pd.read_csv("../data/global_temperature_time_series.csv")

In [3]:
df.head()

,Source,Year,Mean
0,gcag,1850-01,-0.6746
1,gcag,1850-02,-0.3334
2,gcag,1850-03,-0.5913
3,gcag,1850-04,-0.5887
4,gcag,1850-05,-0.5088


In [4]:
df = df.rename({'Source': 'source', 'Year': 'time', 'Mean': 'temperature'}, axis=1)
df['time'] = pd.to_datetime(df['time'])

## Class GroupedLagTimeFeatureCreator

In [5]:
df = df.sample(frac=1)
df.head()

,source,time,temperature
1135,gcag,1912-04-01,-0.3377
586,GISTEMP,1889-06-01,-0.1000
2853,gcag,1983-11-01,0.2455
3625,gcag,2016-01-01,1.0878
3747,gcag,2021-02-01,0.5646


In [6]:
s = GroupedLagTimeFeatureCreator(windows=[2, 3, 4, 5, 6], functions=['mean', 'median', 'max', 'min', 'sum', 'std', 'diff', 'slope'])
df = s.create(df, time='time', groupby_col='source', target='temperature')

In [7]:
df['source'].unique()

array(['GISTEMP', 'gcag'], dtype=object)

In [8]:
df.loc[df['source'] == 'gcag', :].tail(10)

,source,time,temperature,temperature_sum_1_lag,temperature_mean_2_lag,temperature_mean_3_lag,temperature_mean_4_lag,temperature_mean_5_lag,temperature_mean_6_lag,temperature_median_2_lag,...,temperature_diff_lag_2_vs_3,temperature_diff_lag_2_vs_4,temperature_diff_lag_2_vs_5,temperature_diff_lag_2_vs_6,temperature_diff_lag_3_vs_4,temperature_diff_lag_3_vs_5,temperature_diff_lag_3_vs_6,temperature_diff_lag_4_vs_5,temperature_diff_lag_4_vs_6,temperature_diff_lag_5_vs_6
3811,gcag,2023-10-01,1.2866,1.3522,1.27575,1.233833,1.188325,1.12494,1.092033,1.27575,...,0.0493,0.1475,0.3279,0.2718,0.0982,0.2786,0.2225,0.1804,0.1243,-0.0561
3813,gcag,2023-11-01,1.3338,1.2866,1.31940,1.279367,1.247025,1.20798,1.151883,1.31940,...,0.1529,0.2022,0.3004,0.4808,0.0493,0.1475,0.3279,0.0982,0.2786,0.1804
3815,gcag,2023-12-01,1.2586,1.3338,1.31020,1.324200,1.292975,1.26438,1.228950,1.31020,...,-0.0656,0.0873,0.1366,0.2348,0.1529,0.2022,0.3004,0.0493,0.1475,0.0982
3816,gcag,2024-01-01,1.1516,1.2586,1.29620,1.293000,1.307800,1.28610,1.263417,1.29620,...,0.0472,-0.0184,0.1345,0.1838,-0.0656,0.0873,0.1366,0.1529,0.2022,0.0493
3817,gcag,2024-02-01,1.2902,1.1516,1.20510,1.248000,1.257650,1.27656,1.263683,1.20510,...,-0.0752,-0.0280,-0.0936,0.0593,0.0472,-0.0184,0.1345,-0.0656,0.0873,0.1529
3818,gcag,2024-03-01,1.2515,1.2902,1.22090,1.233467,1.258550,1.26416,1.278833,1.22090,...,-0.1070,-0.1822,-0.1350,-0.2006,-0.0752,-0.0280,-0.0936,0.0472,-0.0184,-0.0656
3819,gcag,2024-04-01,1.2053,1.2515,1.27085,1.231100,1.237975,1.25714,1.262050,1.27085,...,0.1386,0.0316,-0.0436,0.0036,-0.1070,-0.1822,-0.1350,-0.0752,-0.0280,0.0472
3820,gcag,2024-05-01,1.0745,1.2053,1.22840,1.249000,1.224650,1.23144,1.248500,1.22840,...,-0.0387,0.0999,-0.0071,-0.0823,0.1386,0.0316,-0.0436,-0.1070,-0.1822,-0.0752
3821,gcag,2024-06-01,1.1154,1.0745,1.13990,1.177100,1.205375,1.19462,1.205283,1.13990,...,-0.0462,-0.0849,0.0537,-0.0533,-0.0387,0.0999,-0.0071,0.1386,0.0316,-0.1070
3822,gcag,2024-07-01,1.1398,1.1154,1.09495,1.131733,1.161675,1.18738,1.181417,1.09495,...,-0.1308,-0.1770,-0.2157,-0.0771,-0.0462,-0.0849,0.0537,-0.0387,0.0999,0.1386


In [11]:
np.mean([1.1154, 1.0745])

np.float64(1.0949499999999999)

In [10]:
df.isna().mean()

source                         0.000000
time                           0.000000
temperature                    0.000000
temperature_sum_1_lag          0.000523
temperature_mean_2_lag         0.001046
                                 ...   
temperature_diff_lag_3_vs_5    0.002616
temperature_diff_lag_3_vs_6    0.003139
temperature_diff_lag_4_vs_5    0.002616
temperature_diff_lag_4_vs_6    0.003139
temperature_diff_lag_5_vs_6    0.003139
Length: 69, dtype: float64